In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# 载入数据集
minist = input_data.read_data_sets('MNIST_data',one_hot=True)

In [ ]:
def RNN(X,weights,biases):
    
    # LSTM参数问题
    # https://blog.csdn.net/wjc1182511338/article/details/79689409
    # input = [batch_size,max_time,n_inputs]
    
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    
    # lstm_size, 个人理解为，a,c这些隐藏单元权重的的维度
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=lstm_size)
    
    # https://www.cnblogs.com/wzdLY/p/10071962.html
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    
    return results

In [ ]:
# 输入图片是 28×28

n_inputs = 28    #输入一行，一行有28个数据

max_time = 28   #一共28行
lstm_size = 100 #隐层单元

n_class = 10    #10分类
batch_size = 50  #没批次50个样本

n_batch = minist.train.num_examples // batch_size

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

weights = tf.Variable(tf.truncated_normal([lstm_size,n_class],stddev=0.1))
biases = tf.Variable(tf.constant(0.1,shape=[n_class]))

#tf.summary.scalar('weights', weights)  # 必须是一个标量
#tf.summary.scalar('biases', biases)

# 定义RNN网络

# 计算RNN返回结果
prediction = RNN(x,weights,biases)

# 损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))

tf.summary.scalar('cross_entropy', cross_entropy)
#优化
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))


accuracy= tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

tf.summary.scalar('accuracy', accuracy)

# summaries合并
merged = tf.summary.merge_all()


my_count = 0

#初始化

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    # 写到指定的磁盘路径中
    train_writer = tf.summary.FileWriter('train/logs', sess.graph)

    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            my_count = my_count + 1           
            bactch_xs,batch_ys = minist.train.next_batch(batch_size)
            summary,_ =sess.run([merged,train_step],feed_dict={x:bactch_xs,y:batch_ys})
            
        
        
        acc = sess.run(accuracy,feed_dict={x:minist.test.images,y:minist.test.labels})
        print("Iter"+str(epoch)+",Testting Accuracy="+str(acc))
        
        train_writer.add_summary(summary, my_count)   # 将训练过程中的数据保存
    
    train_writer.close() 